In [ ]:
# Jupyter notebook for analyzing DOGAMI data, see Scott Tse's emergence-response notebook at
# https://github.com/hackoregon/emergency-response/blob/analytics/notebooks/census_eda_geo.ipynb
# NOTE: Don't need all of these!
# Import modules included in "kitchen-sink"
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import gdal
import pandas as pd
import psycopg2
import seaborn as sns
import sys
# Import modules NOT included in "kitchen-sink", not sure about osgeo...
from dotenv import load_dotenv, find_dotenv
import geoplot as gplt
from osgeo import osr, ogr 
#import rasterio
from rasterstats import zonal_stats

%matplotlib inline

F.D. Pearce, 04/16/18

Notebook for computing statistics on raster pixel values contained within a geometry (shape) file

In [53]:
# Define ALL parameters in dictionary (convert to json config file!)
params = {
    'raster': {
        'root': './CSZ_M9p0_',
        'names': ['pgv_site', 'PGD_landslide_dry', 'PGD_landslide_wet', 'PGD_liquefaction_wet'],
        'ext': '.tif'
    },
    'geometry': {
        'from_postgis': {
            'query': {
                'table_name': 'neighborhood_units',
                'geometry_column': 'wkb_geometry',
                'epsg_code': 4326
            }
        }
    },
    'zonal_stats': {
        'layer': 1,
        'stats': ['count', 'min', 'max', 'mean', 'std']
        
    },
    'stats_classification': {
        'stats_to_class': ['min', 'max', 'mean'],
        'pgv_site': {
            'levels': [-9999, 0.1, 1.1, 3.4, 8.1, 16, 31, 60, 116, 9999],
            'level_labels': ['I', 'II-III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X'],
            'class_name': 'Modified Mercalli Intensity',
            'class_tag': 'MMI'
        },
        'PGD_landslide_dry': {
          'levels': [-9999, 0, 10, 30, 100, 9999],
          'level_labels': ['None', 'Low', 'Moderate', 'High', 'Very High'],
          'class_name': 'Landslide Intensity (Dry)',
          'class_tag': 'DefInt'
        },
        'PGD_landslide_wet': {
          'levels': [-9999, 0, 10, 30, 100, 9999],
          'level_labels': ['None', 'Low', 'Moderate', 'High', 'Very High'],
          'class_name': 'Landslide Intensity (Wet)',
          'class_tag': 'DefInt'
        },
        'PGD_liquefaction_wet': {
          'levels': [-9999, 0, 10, 30, 100, 9999],
          'level_labels': ['None', 'Low', 'Moderate', 'High', 'Very High'],
          'class_name': 'Liquefaction Intensity (Wet)',
          'class_tag': 'DefInt'
        }
    }
}

In [54]:
# Functions for interacting with PostGres database
def pgconnect():
    """Establish connection to PostGres database using the parameters specified in .env file.
    First, walk root diretory to find and load .env file w/ PostGres variables defining database, 
    user, host, password, and port variables.
    Then, return connection to database from psycopg2.connect
    """
    try:
        load_dotenv(find_dotenv())
        conn = psycopg2.connect(database=os.environ.get("PG_DATABASE"), user=os.environ.get("PG_USER"), 
                            password = os.environ.get("PG_PASSWORD"), 
                            host=os.environ.get("PG_HOST"), port=os.environ.get("PG_PORT"))
        print("Opened database successfully\n")
        return conn
    except psycopg2.Error as e:
        print("I am unable to connect to the database\n")
        print(e)
        print(e.pgcode)
        print(e.pgerror)
        print(traceback.format_exc())
        return None

def get_query_string(table_name, geometry_column, epsg_code):
    '''Build query string from parameter inputs defining table name, geometry column
    name, and epsg code.
    '''
    query_string = 'SELECT ST_TRANSFORM({}, {}) AS geometry FROM {}'.format(
            geometry_column, epsg_code, table_name
    )
    return query_string
    
def get_geometry_from_postgis(postgis_params):
    '''
    This function takes a dictionary containing parameters for building a SQL query,
    as defined in get_query_string, then connects to a postgis db, selects the 
    data specified in the query, and finally returns a geodataframe with a single
    column named geometry that contains shape data.
    '''
    query_string = get_query_string(**postgis_params['query'])
    conn = pgconnect()
    #cur = conn.cursor()
    print("SQL QUERY = "+query_string+'\r\n')
    try:
        geo_df = gpd.GeoDataFrame.from_postgis(
            query_string, 
            conn, 
            geom_col='geometry', 
            crs={'init': u'epsg:{}'.format(postgis_params['query']['epsg_code'])}, 
            coerce_float=False
        )
        return geo_df
    except Exception as e:
        print(e)
    finally:
        conn.close()

In [55]:
# Define functions for manipulating geoshapes and raster files
def get_gdfcrs_epsg(gdf):
    """Return integer EPSG code corresponding to Coordinate Reference
    used in input geodataframe, gdf. Attribute gdf.crs must contain
    a dict with key = 'init' that contains a string starting with 'epsg',
    followed by a colon, followed by an integer as a string.
    """
    try:
        dfepsg = gdf.crs['init'].split(':')
        if dfepsg[0] == 'epsg':
            return int(dfepsg[1])
    except:
        print('Error: geodataframe crs = {}, unrecognized EPSG integer'.format(gdf.crs['init']))

def get_raster_info_srs(raster_file, print_info=True):
    """Print information about raster file, and return its
    spatial reference system using gdal.
    """
    if print_info:
        try:
            print(gdal.Info(raster_file))
        except:
            print("Error reading info from raster file = {}".format(raster_file))
    try:
        raster = gdal.Open(raster_file)
    except:
        print("Error opening raster file = {}".format(raster_file))
    else:
        raster_srs = osr.SpatialReference()
        raster_srs.ImportFromWkt(raster.GetProjection())
        return raster_srs
    
def convert_polygons_from_srsinp_to_srsout(geom_col, srs_out):
    """Transform list of georeferenced polygon geometries from geopandas
    dataframe geometry column, geom_col, to the desired output Spatial Reference, srs_out.
    The input geometry column, geom_col, MUST have a valid epsg code defining its Spatial 
    Reference (SRS)."""
    # Define input spatial reference using epsg code from gdf
    srs_inp = ogr.osr.SpatialReference()
    srs_inp.ImportFromEPSG(get_gdfcrs_epsg(geom_col))
    poly_out = []
    # Define list of polygons in transformed spatial reference
    for g in geom_col:
        # If MultiPolygon, then assume it contains only one Polygon
        if g.type == 'MultiPolygon':
            poly = ogr.CreateGeometryFromWkt(g.geoms[0].wkt)
        elif g.type == 'Polygon':
            # Need to test this
            poly = ogr.CreateGeometryFromWkt(g.geoms.wkt)
        else:
            print("Error: geometry = {}, MUST be Polygon or MultiPolygon".format(g.type))
        poly.AssignSpatialReference(srs_inp)
        # Convert point co-ordinates so that they are in same projection as raster 
        poly.Transform(osr.CoordinateTransformation(srs_inp, srs_out))
        poly_out.append(poly.ExportToWkt())
    return poly_out

In [56]:
# Step 1) Select geometry column either from Postgis db (implemented), or
# from shapefile (not yet implemented).  In eithe case, make sure geometry
# has a valid epsg Spatial reference assigned to it, such as 4326 (lon/lat)
# For a Postgis-derived geometry, this is done on the db-side using ST_TRANSFORM
gdf = get_geometry_from_postgis(params['geometry']['from_postgis'])
gdf.info()
print(gdf['geometry'][0].type)
print(gdf['geometry'].crs)

Opened database successfully

SQL QUERY = SELECT ST_TRANSFORM(wkb_geometry, 4326) AS geometry FROM neighborhood_units

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 1 columns):
geometry    876 non-null object
dtypes: object(1)
memory usage: 6.9+ KB
MultiPolygon
{'init': 'epsg:4326'}


In [57]:
# Step 2) Print info about tif file and get its spatial reference info
raster_name = params['raster']['names'][0]
raster_file = params['raster']['root'] + raster_name + params['raster']['ext']
srs_raster = get_raster_info_srs(raster_file, print_info=False)


In [58]:
# Step 3) Generate a list of point values transformed from the srs used in the 
# input geodataframe, gdf, to the srs used in the raster file, srs_raster
# This is inefficient and not very general, better to do handle with postgis in db query,
# or just fix this up?
geom_ras = convert_polygons_from_srsinp_to_srsout(gdf['geometry'], srs_raster)
#geom_ras = get_geopoints_gpdepsginp_to_srsout( \
#        geom_type, gdf, 'geometry', None, srs_raster
#)
print(len(geom_ras))
print(len(gdf))

876
876


In [59]:
# Step 4) Use rasterstats to compute analytics on pixel values within specified geometry, 
# MUST be polygon or multipolygon and transformed to srs_raster!
# Add stats from pixel values into geodataframe that defines geometry
geomstats = zonal_stats(geom_ras, raster_file, **params['zonal_stats'])
gdf_pv = pd.DataFrame(geomstats, index=gdf.index)
gdf_pv.rename(columns={co: raster_name+'_'+co for co in gdf_pv.columns}, inplace=True)
gdf_merge = pd.merge(gdf, gdf_pv, left_index=True, right_index=True)
#print(nustats_pgv)

/home/jupyter/.conda/envs/jupyter/lib/python3.6/site-packages/rasterstats/main.py:145: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  with Raster(raster, affine, nodata, band) as rast:
/home/jupyter/.conda/envs/jupyter/lib/python3.6/site-packages/rasterstats/io.py:242: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  self.affine = guard_transform(self.src.transform)
/home/jupyter/.conda/envs/jupyter/lib/python3.6/site-packages/rasterstats/main.py:165: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  np.issubdtype(fsrc.array.dtype, float)


In [60]:
# Step 5) Classify a subset of the geometry statistics, converting the calculated
# stat in pixel values to a label describing the stats intensity bin
print(gdf_merge.columns)
gdf_merge.info()



Index(['geometry', 'pgv_site_count', 'pgv_site_max', 'pgv_site_mean',
       'pgv_site_min', 'pgv_site_std'],
      dtype='object')
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 6 columns):
geometry          876 non-null object
pgv_site_count    876 non-null int64
pgv_site_max      876 non-null float64
pgv_site_mean     876 non-null float64
pgv_site_min      876 non-null float64
pgv_site_std      876 non-null float64
dtypes: float64(4), int64(1), object(1)
memory usage: 41.1+ KB


In [68]:
# Classify specified raster statistics contained in dataframe column(s)
#'stats_classification': {
#        'stats_to_class': ['min', 'max', 'mean'],
#        'pgv_site': {
#            'levels': [0.1, 1.1, 3.4, 8.1, 16, 31, 60, 116],
#            'level_labels': ['I', 'II-III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X'],
#            'class_name': 'Modified Mercalli Intensity',
#            'class_tag': 'MMI'
#        },

#pd.cut(df.value, range(0, 105, 10), right=True, labels=labels)

def get_stats_classification(gdf, **kwargs):
    """Classify raster statistics using specified parameters in kwargs"""
    raster_names = [rn for rn in kwargs.keys() if rn != "stats_to_class"]
    for rn in raster_names:
        stats_to_class = [rn+'_'+sc for sc in kwargs['stats_to_class']]
        levels = kwargs[rn]['levels']
        labels = kwargs[rn]['level_labels']
        ctag = '_' + kwargs[rn]['class_tag']
        for s2c in stats_to_class:
            try:
                gdf[s2c+ctag] = pd.cut(gdf[s2c], levels, right=True, labels=labels)
            except KeyError:
                print("Key Error exception occurred for raster stat key = {}".format(s2c))
    return gdf

if 'stats_classification' in params:
    gdf_merge_class = get_stats_classification(gdf_merge, **params['stats_classification'])
print(gdf_merge_class.info())

Key Error exception occurred for raster stat key = PGD_landslide_dry_min
Key Error exception occurred for raster stat key = PGD_landslide_dry_max
Key Error exception occurred for raster stat key = PGD_landslide_dry_mean
Key Error exception occurred for raster stat key = PGD_landslide_wet_min
Key Error exception occurred for raster stat key = PGD_landslide_wet_max
Key Error exception occurred for raster stat key = PGD_landslide_wet_mean
Key Error exception occurred for raster stat key = PGD_liquefaction_wet_min
Key Error exception occurred for raster stat key = PGD_liquefaction_wet_max
Key Error exception occurred for raster stat key = PGD_liquefaction_wet_mean
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 9 columns):
geometry             876 non-null object
pgv_site_count       876 non-null int64
pgv_site_max         876 non-null float64
pgv_site_mean        876 non-null float64
pgv_site_min         876 non-null float64
pgv_site_std

### Plotting

In [ ]:
# Make a map showing distribution of df column, col2plot_y
#col2plot_y = 'casualties_avetotal'
#col2plot_y = 'buildingloss'
#col2plot_y = 'buildingloss_ratio'
col2plot_y = 'nupv_mean'

gplt.choropleth(gdf_merge,
                hue=gdf_merge[col2plot_y],  # Display data, passed as a Series
                projection=gplt.crs.AlbersEqualArea(),
                cmap='Purples',
                linewidth=0.5,
                edgecolor='black',
                k=5,
                legend=True,
                scheme='equal_interval',
                figsize=(12, 12)
)
plt.title("{}".format(col2plot_y), fontsize=24)